<a href="https://colab.research.google.com/github/ChristianKitte/HelloCodeCleaning/blob/main/EA_6_Data_Cleaning_Excercise_Aufgabe_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Aufgabe:**

Inhalt dieser Einsendeaufgabe ist das Cleanen einer kleineren CSV Datei. Über den Kontext der Datei ist nichts bekannt. Inhalte können somit nicht anhand der Verwendung oder Herkunft semantisch beurteilt werden. 

In diesem konkreten Fall bedeutet dies, dass ein Alter von 4 Jahren durchaus berechtigt sein kann. 

Zudem scheint es bei der Datei um eine allein stehende Datei zu handeln, sprich, deren Werte stehen nicht direkt mit anderen Dateien in Verbindung.

In diesem konkreten Fall bedeutet dies, dass die Angabe einer ID keine zusätlichen Informationen enthält.

Grundsätzlich ist zu klären, ob Datensätze mit fehlenden Werten wirklich valide sind, sprich welche Felder tatsächlich leer sein dürfen oder nur fehlerhaft leer sind. Dies ist aber vom konkreten Fall abhängig.

*(Im folgenden steht die DS für Datensatz resp. Datensätze)*

In [8]:
# Bibliotheken importieren
import pandas as pd

# https://pypi.org/project/email-validator/
%pip install email-validator
from email_validator import validate_email, EmailNotValidError

print("Fertig...")

Fertig...


Ein erster Blick kann einen Überblick über die Anzahl der nicht belegten Werte sowie belegten Werte geben. Dies gibt einen (wirklich nur) ersten Eindruck darüber, wie der Datensatz zu bewerten ist.



In [9]:
# Originaldaten laden und anschauen - Teil 1
url="https://raw.githubusercontent.com/edlich/eternalrepo/master/DS-WAHLFACH/dsm-beuth-edl-demodata-dirty.csv"
ds= pd.read_csv(url)
#ds.head()
ds.tail()
#ds

,id,full_name,first_name,last_name,email,gender,age
18,19.0,Clair Skillern,Clair,Skillern,cskillerng@nih.gov,Male,-78
19,20.0,Mathew Addicott,Mathew,Addicott,maddicotth@acquirethisname.com,Male,65
20,21.0,Kerianne Goacher,Kerianne,Goacher,NaN,Female,45
21,NaN,Maurits Shawl,Maurits,Shawl,mshawlj@dmoz.org,Male,72
22,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Originaldaten laden und anschauen - Teil 2
print("Nicht NaN")
print(ds.count())
print()
print("NaN")
print(ds.isnull().sum())

Nicht NaN
id            20
full_name     21
first_name    21
last_name     21
email         20
gender        20
age           21
dtype: int64

NaN
id            3
full_name     2
first_name    2
last_name     2
email         3
gender        3
age           2
dtype: int64


**Erster Eindruck:**

**1)**
Der Datensatz enthält eine ID. Auf Grund der zu Anfang gemachten Erläuterung enthält diese keine zusätzliche Information. 

**2)**
Der Datensatz enthält Spalten mit Vor- und Nachnamen sowie eine Konkatenation von beiden. 

 
Dies ist mindestens aus zwei Gründen nachteilig: 

> **a)** Es kann nicht garantiert werden, dass die Zusammenführung fehlerfrei ist (hier vielleicht, aber nicht bei beispielsweise 80000 DS). Im Zweifelsfall sollte man sie lieber selbst implementieren.

> **b)** Zum anderen handelt es sich um redundante Informationen (welche ggf. eine beachtliche Menge an Speicher gebrauchen und Verarbeitungsschritte verlangsammen können). Kummulierte Werte können im Rahmen einer späteren Nutzung durchaus sinnvoll sein, jedoch nicht bei der Aufbereitung und Bereitstellung von Basisdaten. 

Da aus den getrennten Eigenschaften mehr Information (zum Beispiel wie in meiner Firma Rückschlüsse auf das Geburtsjahr durch den Vornamen und Toplisten der Lieblingsnahmen für jedes Jahr) extrahiert werden kann, sehe ich die Nutzung der getrennten Namen als sinnvoller an.

**Fazit:**
Für die weitere Verarbeitung verwerfe ich die Spalten "ID" und "full_name". Hierdurch werden die weiteren Schritte beschleunigt (zumindest wenn ich einen wirklich großen DS hätte) und mögliche Inkonsistenzen in diesen Spalten entfallen. 

In [11]:
ds = ds[["first_name", "last_name", "email", "gender", "age"]]
ds_save=ds.copy(deep=True)
#ds.head()
ds.tail()
#ds

,first_name,last_name,email,gender,age
18,Clair,Skillern,cskillerng@nih.gov,Male,-78
19,Mathew,Addicott,maddicotth@acquirethisname.com,Male,65
20,Kerianne,Goacher,NaN,Female,45
21,Maurits,Shawl,mshawlj@dmoz.org,Male,72
22,NaN,NaN,NaN,NaN,NaN


**Spalte Alter:**

Hier werden die Nullwerte durch "0" ersetzt, der Typ in Integer umgewandelt und negative Werte auf 0 gesetzt.

In [12]:
ds["age"].unique()

# ==> old, NaN offensichtlich falsch

ds["age"]=ds["age"].fillna("0")
ds["age"]=ds["age"].replace("old","0")

# ==> umwandeln in Integer
ds=ds.astype({"age":int})

# ==> Werte kleiner 1?

# boolsche Indizierung
mask= ds["age"]<0
werte_kleiner_null=ds[mask]
#print("Werte kleiner 0: ", werte_kleiner_null)

# ==> Wert 78 ersetzen
ds["age"]=ds["age"].replace(-78,0)

#print(ds["age"])
#print(ds_save["age"])

print("Spalte Alter ist bereinigt...")

Spalte Alter ist bereinigt...


**Spalte Geschlecht:**

Die Angabe erfolgt textlich als Male und Female. Es existieren Nullwerte, welche hier auf definiert "unknown" gesetzt werden. 

In der Praxis könnte könnte für das Merkmal "Geschlecht" hier ein Abgleich mit einer Namensliste erfolgen und anhand dessen eine wahrscheinliche Einteilung erfolgen. Dies funktioniert in der Praxis ziemlich genau. 

Zudem sollte das Geschlecht statt mit Texten lieber mit Integerwerten vercodet werden. Vergleiche laufen schneller und es kann zu keinen Tippfehlern kommen.

In [13]:
ds["gender"].unique()

# ==> Es existieren Nullwerte. Diese auf unknown setzen 
ds["gender"]=ds["gender"].fillna("unknown")

ds["gender"].unique()

# ==> Spalte für Vercodung erstellen und vercoden
ds["code_gender"]=ds["gender"]
ds["code_gender"]=ds["code_gender"].replace({"Female":1, "Male":2, "unknown":3})

# ==> Das alte DataFrame bereinigen
ds = ds[["first_name", "last_name", "email", "code_gender", "age"]]

# ==> umwandeln in Integer
ds=ds.astype({"code_gender":int})

#print(ds["code_gender"])

print("Spalte Geschlecht ist bereinigt...")
print("Codeliste: Female:1, Male:2, unknown:3")

Spalte Geschlecht ist bereinigt...
Codeliste: Female:1, Male:2, unknown:3


**Spalte first_name, last_name und mail:**

Die Möglichkeiten für eine Fehlerkorrektur sind hier Grenzen gesetzt, da man zum einen nicht auf weitergehende Daten zugreifen kann (es existiert ja nur diese eine Datei), zum anderen aber Namen von Personen sehr unterschiedlich sein können, ebenso wie die von Mailadressen (beispielsweise bei der Schreibung).

Daher gehe ich hier sehr vorsichtig heran und lösche zunächst lediglich die leeren Felder und schreibe hier definiert "unknown" hinein, da es in der Regel einfacher ist, einen festen Wert zu haben, als gar nichts (so etwas führt schnell mal zu einem Fehler :)  ).

Die separate Prüfung des Vor- und Nachnamens auf Duplikate ist mit Vorsicht zu genießen. Aus Erfahrung weiß ich, dass bei großen Datenbeständen gleiche Namenspaare eher normal als ungewöhnlich sind.

Interessant wird das Hinzuziehen der eMail. Diese Prüfe ich zunächst auf die Einhaltung eines typischen Formats und ersetze die ungültigen ebenfalls durch "unknown".

Im letzten Schritt prüfe ich auf Gleichheit oder Fehlen aller drei Spalten. Im ersten Fall ist es anhand der vorliegenden Daten ein identischer DS (es kann sich real vielleicht um eine unterschiedliche Person und falsche Eingaben handeln, aber anhand des DS ist er identisch und nur eine redundante Information). Daher lösche ich diese bis auf einen DS. Im zweiten Fall enthält ein solcher DS keine Information. Daher werden diese DS ebenfalls gelöscht.

In [14]:
# ==> Teil 1: Es existieren Nullwerte. Diese auf unknown setzen 
ds["first_name"]=ds["first_name"].fillna("unknown")
ds["last_name"]=ds["last_name"].fillna("unknown")
ds["email"]=ds["email"].fillna("unknown")
ds

,first_name,last_name,email,code_gender,age
0,Mariel,Finnigan,mfinnigan0@usda.gov,1,60
1,Kenyon,Possek,kpossek1@ucoz.com,2,12
2,Lalo,Manifould,lmanifould2@pbs.org,2,26
3,Nickola,Carous,ncarous3@phoca.cz,2,4
4,Norman,Dubbin,ndubbin4@wikipedia.org,2,17
5,Hasty,Perdue,hperdue5@qq.com,3,77
6,Franz,Castello,fcastello6@1688.com,2,25
7,Jorge,Tarney,jtarney7@ft.com,2,77
8,Eunice,Blakebrough,eblakebrough8@sohu.com,1,45
9,Kristopher,Frankcombe,kfrankcombe9@slate.com,2,0


In [15]:
# ==> Teil 2: Falsche eMails auf "unkown" setzen
#false_email_adrs = ds[verify_email(ds["email"])==True]

def checkMail(email):
  try:
    # Validate.
    valid = validate_email(email)
    return True 
    # Update with the normalized form.
    #email = valid.email
  except EmailNotValidError as e:
    # email is not valid, exception message is human-readable
    return False
    #print(str(e))


ds["valid_mail"] = [False if checkMail(x) == False else True for x in ds["email"]]
ds

,first_name,last_name,email,code_gender,age,valid_mail
0,Mariel,Finnigan,mfinnigan0@usda.gov,1,60,True
1,Kenyon,Possek,kpossek1@ucoz.com,2,12,True
2,Lalo,Manifould,lmanifould2@pbs.org,2,26,True
3,Nickola,Carous,ncarous3@phoca.cz,2,4,True
4,Norman,Dubbin,ndubbin4@wikipedia.org,2,17,True
5,Hasty,Perdue,hperdue5@qq.com,3,77,True
6,Franz,Castello,fcastello6@1688.com,2,25,True
7,Jorge,Tarney,jtarney7@ft.com,2,77,True
8,Eunice,Blakebrough,eblakebrough8@sohu.com,1,45,True
9,Kristopher,Frankcombe,kfrankcombe9@slate.com,2,0,True


In [16]:
# ==> Teil 3: Löschen der Spalten mit Vorname=Nachname=email="unknown"

ds["remove"]=(ds["first_name"]=="unknown") & (ds["last_name"]=="unknown") & (ds["email"]=="unknown")
ds=ds.where(ds["remove"]== False)
ds=ds.dropna()

ds=ds.drop(["valid_mail", "remove"],axis=1)

ds=ds.astype({"age":int})
ds=ds.astype({"code_gender":int})

print("Die Aufgabe ist erledigt:")

ds

Die Aufgabe ist erledigt:


,first_name,last_name,email,code_gender,age
0,Mariel,Finnigan,mfinnigan0@usda.gov,1,60
1,Kenyon,Possek,kpossek1@ucoz.com,2,12
2,Lalo,Manifould,lmanifould2@pbs.org,2,26
3,Nickola,Carous,ncarous3@phoca.cz,2,4
4,Norman,Dubbin,ndubbin4@wikipedia.org,2,17
5,Hasty,Perdue,hperdue5@qq.com,3,77
6,Franz,Castello,fcastello6@1688.com,2,25
7,Jorge,Tarney,jtarney7@ft.com,2,77
8,Eunice,Blakebrough,eblakebrough8@sohu.com,1,45
9,Kristopher,Frankcombe,kfrankcombe9@slate.com,2,0


Ein abschließender Blick zeigt, dass es keine nicht belegten Werte im Sinne von NaN gibt. Durch die Belegung mit "unknown" haben wir einen klar definierten Wert für "nicht belegt". 

Zudem haben wir nun schöne, performante int64 Werte für die weitere Arbeit.

Sicher ist "unknown" immer noch unbekannt, aber er wurde explizit angeschaut, bewertet und auf einen dafür vorgesehenen Wert gesetzt.

In [17]:
# Abschließender Blick

print("Nicht NaN")
print(ds.count())
print()
print("NaN")
print(ds.isnull().sum())

Nicht NaN
first_name     21
last_name      21
email          21
code_gender    21
age            21
dtype: int64

NaN
first_name     0
last_name      0
email          0
code_gender    0
age            0
dtype: int64
